## Create Base Portfolios

This notebook creates Base Portfolios in Moody's Risk Modeler

In [ ]:
%load_ext autoreload
%autoreload 2

# Import required modules
import sys
import os
from pathlib import Path

# Determine the notebook's actual directory
cwd = Path.cwd()

if 'Active_' in str(cwd):
    # Working directory is set correctly, construct path to THIS notebook
    notebook_path = cwd / 'Step_02_Create_Base_Portfolios.ipynb'
else:
    # Working directory is not set correctly (e.g., /home/jovyan)
    home = Path.home()
    workspace = home / 'workspace'
    
    if workspace.exists():
        workflows = workspace / 'workflows'
        active_dirs = list(workflows.glob('Active_*'))
        
        if active_dirs:
            # Use the first Active_ directory found and point to THIS notebook
            notebook_path = active_dirs[0] / 'notebooks' / 'Stage_03_Data_Import' / 'Step_02_Create_Base_Portfolios.ipynb'
        else:
            raise RuntimeError("No Active_ cycle directory found in workspace/workflows/")
    else:
        raise RuntimeError("Workspace directory not found")

print(f"Notebook path: {notebook_path}")

# Add workspace to path
workspace_path = notebook_path.parent.parent.parent.parent
if str(workspace_path) not in sys.path:
    sys.path.insert(0, str(workspace_path))

from helpers import ux
from helpers.context import WorkContext
from helpers.batch import submit_batch, get_batch_jobs, read_batch
from helpers.job import track_job_status
from helpers.database import execute_query
from helpers.step import Step, get_last_step_run
from helpers.irp_integration import IRPClient

In [ ]:
# Initialize work context
ux.header("Base Data Import")

# Initialize context with explicit notebook path
context = WorkContext(notebook_path=str(notebook_path))

ux.info(f"Cycle: {context.cycle_name}")
ux.info(f"Stage: {context.stage_name}")
ux.info(f"Step: {context.step_name}")

# Initialize step execution tracking
step = Step(context)

if step.executed:
    ux.warning("⚠ This step has already been executed")
    ux.info(f"Message: {step.status_message}")
    
    last_run = get_last_step_run(step.step_id)
    if last_run:
        ux.info(f"Last run: #{last_run['run_num']}")
        ux.info(f"Status: {last_run['status']}")
        if last_run['completed_ts']:
            ux.info(f"Completed: {last_run['completed_ts'].strftime('%Y-%m-%d %H:%M:%S')}")
    
    response = ux.yes_no("Do you want to re-run this step?")
    if response:
        ux.info("Re-running step...")
        step.start(force=True)
    else:
        ux.info("Step execution skipped")
        raise SystemExit("Step already completed")

ux.success(f"✓ Step tracking initialized for '{context.step_name}'")